In [ ]:
using BenchmarkTools
using Distributed
gpu = addprocs([("cbauer17@gpu2", 1)]; exename=`/projects/ag-trebst/bauer/bin/julia-1.3.1/bin/julia --project=/projects/ag-trebst/bauer/JuliaOulu20`, dir="/projects/ag-trebst/bauer/JuliaOulu20", tunnel=true)[1]
@fetchfrom gpu DEPOT_PATH[1] = "/projects/ag-trebst/bauer/.julia";
@fetchfrom gpu eval(Meta.parse("using CuArrays, BenchmarkTools"));

In [ ]:
@everywhere begin
    using Statistics
    
    function bench(A,B)
        ts = zeros(100) # in ms
        for i in eachindex(ts)
            t = time();
            A*B;
            ts[i] = (time()-t) * 1e3
        end
        minimum(ts)
    end
end

In [ ]:
@fetchfrom gpu begin
    A, B = rand(1000,1000), rand(1000,1000);
    Agpu, Bgpu = CuArray(A), CuArray(B);
    
    println("A*B (cpu)")
    A*B; # warmup
    t = bench(A,B)
    println(round(t, digits=2), " ms")

    println("A*B (gpu)")
    Agpu*Bgpu; # warmup
    t = bench(Agpu,Bgpu)
    println(round(t * 1e3, digits=2), " μs")
end